In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [3]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s41.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
41-24887,0,0,6,2,2,5.0,77,70.0,1,132.0,...,0,0,0,0,0,0,27,22,1,1
41-17428,0,0,12,1,3,4.0,67,90.0,1,121.0,...,0,0,0,0,0,0,26,23,1,1
41-06946,0,0,7,1,2,5.0,84,60.0,77,126.0,...,0,0,0,0,0,0,34,30,1,1
41-17653,0,0,9,2,3,3.3,71,70.0,1,134.0,...,0,0,0,0,0,0,25,31,1,1
41-17118,0,0,8,8,8,3.3,82,80.0,77,120.0,...,0,0,0,0,0,0,29,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41-36132,0,0,5,3,2,6.7,87,100.0,77,151.0,...,0,0,0,0,0,0,31,33,1,1
41-42972,1,0,9,2,3,2.9,66,100.0,77,126.0,...,0,0,0,0,0,0,21,24,1,1
41-39190,0,0,11,2,3,4.4,83,90.0,77,130.0,...,0,0,0,0,0,0,26,23,1,1


In [5]:
X.shape

(17785, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [10]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(12449, 195)
(17785, 195)
(17785,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [11]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [12]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(624, 195)
(624,)
[312 312]


<h1 align='center'>Basic Gradient Boosting</h1>

In [13]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[4024 1178]
 [  19  115]]
              precision    recall  f1-score   support

           0       1.00      0.77      0.87      5202
           1       0.09      0.86      0.16       134

    accuracy                           0.78      5336
   macro avg       0.54      0.82      0.52      5336
weighted avg       0.97      0.78      0.85      5336

ROC AUC prob1: 0.8788783877613089
MCC: 0.2306943236069416
F2: 0.3143794423182067
PR_AUC: 0.15703770535178238
CPU times: user 344 ms, sys: 2.39 ms, total

In [14]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[4173 1029]
 [  27  107]]
              precision    recall  f1-score   support

           0       0.99      0.80      0.89      5202
           1       0.09      0.80      0.17       134

    accuracy                           0.80      5336
   macro avg       0.54      0.80      0.53      5336
weighted avg       0.97      0.80      0.87      5336

ROC AUC prob1: 0.8707816167145817
MCC: 0.2296046180391226
F2: 0.3199760765550239
PR_AUC: 0.16827930566797322
CPU times: user 4.29 s, sys: 0 ns, total: 4

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [16]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 12:50:17,461] A new study created in memory with name: no-name-dcfd21cc-5846-4455-80f4-e36e0d973af3
[I 2021-12-10 12:50:18,272] Trial 0 finished with value: 0.21643418033075995 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.21643418033075995.


MCC: [0.21385669452465605, 0.22050993055177648, 0.21282253046663716, 0.2185475657799702]
ROC: [0.86422804415884, 0.8683915749659695, 0.8578176792709664, 0.85851077856154]
PR_AUC: [0.10302145474890229, 0.10490676750633003, 0.11784707515351844, 0.12080322240851146]


[I 2021-12-10 12:50:20,041] Trial 1 finished with value: 0.20935055216084775 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.21643418033075995.


MCC: [0.21576177522786175, 0.20995989746238292, 0.2041418226149563, 0.20753871333819007]
ROC: [0.8552315043664525, 0.8578696343402226, 0.8544218959443872, 0.8627278495616059]
PR_AUC: [0.1202600414617948, 0.11168743597214881, 0.12160222738834559, 0.13360998261289672]


[I 2021-12-10 12:50:21,741] Trial 2 finished with value: 0.21976603740907424 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.21976603740907424.


...

MCC: [0.22184421168841112, 0.22319956497422325, 0.21863835926497585, 0.2236286882492857]
ROC: [0.8671877574559236, 0.8688425449671124, 0.8647505637125015, 0.8657768310369833]
PR_AUC: [0.11476464618042123, 0.12240955499814002, 0.12827201136204006, 0.1507440113341566]


[I 2021-12-10 12:50:56,526] Trial 23 finished with value: 0.21175659392136953 and parameters: {'n_estimators': 76, 'subsample': 0.9258785527064558, 'max_depth': 17, 'max_features': 10, 'learning_rate': 0.18686415613602847, 'min_samples_leaf': 44}. Best is trial 19 with value: 0.22632206881548675.


MCC: [0.20662854381538343, 0.2240809608467469, 0.19691899889052125, 0.21939787213282658]
ROC: [0.8546383259186027, 0.8642725770753452, 0.8493697850099236, 0.8652535598918848]
PR_AUC: [0.11870745940563113, 0.12340919368030653, 0.11725739388507075, 0.144128396296758]


[I 2021-12-10 12:50:57,610] Trial 24 finished with value: 0.217316634191831 and parameters: {'n_estimators': 36, 'subsample': 0.9820726610349131, 'max_depth': 14, 'max_features': 11, 'learning_rate': 0.17154941448458988, 'min_samples_leaf': 32}. Best is trial 19 with value: 0.22632206881548675.


MCC: [0.21674423492223502, 0.222289041869779, 0.2106768847551386, 0.21955637522017143]
ROC: [0.8607184050090625, 0.8622026871161819, 0.8596839053586459, 0.8635477783637682]
PR_AUC: [0.12364355478229892, 0.1144142606527407, 0.12694710534213796, 0.12854678719091228]
CPU times: user 39.7 s, sys: 617 ms, total: 40.3 s
Wall time: 40.2 s


In [21]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
#        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
#        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)
#        model.fit(X_res, y_res)

        weights = class_weight.compute_sample_weight('balanced', y=y_train_c)
        model.fit(X_train_c, y_train_c, sample_weight=weights)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 14:05:55,147] A new study created in memory with name: no-name-7baee9f3-41f1-4913-b5d8-ff14fa2528e0
[I 2021-12-10 14:06:07,610] Trial 0 finished with value: 0.1977736653386975 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.1977736653386975.


MCC: [0.19323489136313596, 0.18292581180981932, 0.20243641044702243, 0.2124975477348123]
ROC: [0.8523685944966222, 0.8658707150056631, 0.8618078285898355, 0.86058223185444]
PR_AUC: [0.1654322753496176, 0.16237035467999034, 0.15750053696079433, 0.17279273479213383]


[I 2021-12-10 14:06:42,680] Trial 1 finished with value: 0.18716216826304696 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.1977736653386975.


MCC: [0.17978494040785276, 0.17387030014318158, 0.19111898086935966, 0.20387445163179382]
ROC: [0.8465294941506014, 0.8258590770701497, 0.8535843802279789, 0.8568070571560419]
PR_AUC: [0.11961352370497694, 0.1354891661743422, 0.13546161461902112, 0.1437961222152409]


[I 2021-12-10 14:07:13,537] Trial 2 finished with value: 0.18843169829788836 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.1977736653386975.


...

MCC: [0.20784069004336825, 0.24529692009604734, 0.22821313457567405, 0.2373079350465172]
ROC: [0.8689034437304333, 0.8818042956451261, 0.8693693693693694, 0.8796363059529304]
PR_AUC: [0.1499140677807782, 0.16275702795109595, 0.14697852255266017, 0.16607390427106491]


[I 2021-12-10 14:23:00,873] Trial 23 finished with value: 0.1047327349929128 and parameters: {'n_estimators': 69, 'subsample': 0.9820369123337096, 'max_depth': 20, 'max_features': 10, 'learning_rate': 0.1712498753449273, 'min_samples_leaf': 6}. Best is trial 10 with value: 0.2354243732956553.


MCC: [0.042203694300907896, 0.19923187646684823, 0.10001426767264698, 0.07748110153124811]
ROC: [0.834223101005108, 0.8464457537121897, 0.856256948990513, 0.8421863669325599]
PR_AUC: [0.12427161763409422, 0.1453184577693277, 0.13406121448469097, 0.11755830202451656]


[I 2021-12-10 14:23:25,909] Trial 24 finished with value: 0.1855152098129284 and parameters: {'n_estimators': 85, 'subsample': 0.9423994435571657, 'max_depth': 5, 'max_features': 11, 'learning_rate': 0.19005020005150144, 'min_samples_leaf': 13}. Best is trial 10 with value: 0.2354243732956553.


MCC: [0.18258113062463469, 0.19145526118489997, 0.17623712071390477, 0.1917873267282743]
ROC: [0.8397367770637666, 0.8518948013757702, 0.8414268940220498, 0.8414406025446635]
PR_AUC: [0.12442800221137888, 0.1223127834575828, 0.12124286029086931, 0.11367359262472913]
CPU times: user 17min 30s, sys: 927 ms, total: 17min 30s
Wall time: 17min 30s


In [23]:
best = study.best_trial
best

FrozenTrial(number=10, values=[0.2354243732956553], datetime_start=datetime.datetime(2021, 12, 10, 14, 14, 23, 289085), datetime_complete=datetime.datetime(2021, 12, 10, 14, 14, 38, 186807), params={'n_estimators': 82, 'subsample': 0.9869602664638994, 'max_depth': 3, 'max_features': 10, 'learning_rate': 0.1867212049748521, 'min_samples_leaf': 4}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=10, state=TrialState.COMPLETE, value=None)

In [24]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[4073 1129]
 [  23  111]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.88      5202
           1       0.09      0.83      0.16       134

    accuracy                           0.78      5336
   macro avg       0.54      0.81      0.52      5336
weighted avg       0.97      0.78      0.86      5336

ROC AUC prob1: 0.8701790930009698
MCC: 0.22647477418563908
F2: 0.31250000000000006
PR_AUC: 0.1603509845415154
CPU times: user 136 ms, sys: 5.02 ms, total: 141 ms
Wall time: 138 ms


In [25]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[4192 1010]
 [  22  112]]
              precision    recall  f1-score   support

           0       0.99      0.81      0.89      5202
           1       0.10      0.84      0.18       134

    accuracy                           0.81      5336
   macro avg       0.55      0.82      0.53      5336
weighted avg       0.97      0.81      0.87      5336

ROC AUC prob1: 0.8770134334096529
MCC: 0.2463780966848265
F2: 0.3377563329312425
PR_AUC: 0.143792394503228
CPU times: user 474 ms, sys: 5.03 ms, total: 479 ms
Wall time: 476 ms


In [26]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delfetalpos     0.208864
Presentdel      0.081449
Delmode         0.060527
Anteanemia      0.058732
MthInd_Miso     0.048175
CS_FTP          0.043437
transfus_yes    0.043320
Lac_Min         0.031142
Admcontract     0.026786
Intrachorio     0.020832
Admcervpos      0.019700
Lac_None        0.019461
AdmDBP          0.018383
Augment         0.017812
Education       0.016026
BESTGA          0.015832
Dilat_lst       0.015541
Analgesia       0.014529
AdmSBP          0.014098
AdmBishop       0.013313
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [28]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode         0.130822
Delfetalpos     0.059513
transfus_yes    0.052151
Anteanemia      0.048770
Presentdel      0.046679
Lac_None        0.039831
new_BMI         0.036245
Admefface       0.035901
BESTGA          0.033457
Analgesia       0.030581
Hxcsection      0.024901
new_age         0.024420
Dilat_lst       0.022652
AdmSBP          0.022127
MthInd_Mec      0.021862
CS_FTP          0.021307
chorio          0.018980
MthInd_Miso     0.015475
HosEpitype      0.015274
AdmDBP          0.014916
dtype: float64

In [20]:
gb_params # 5 runs

{'n_estimators': 82,
 'subsample': 0.813187269381181,
 'max_depth': 8,
 'max_features': 20,
 'learning_rate': 0.10560624242958987,
 'min_samples_leaf': 23,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [23]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>